# Gaussian Process Surrogate — From Prior to Posterior

**Bayesian Optimisation Series · Notebook 1 of 3**

This notebook implements a Gaussian Process from scratch using only NumPy, then demonstrates:
1. Sampling functions from the GP prior
2. Conditioning on observations to get the posterior
3. How uncertainty collapses near observed data
4. The effect of noise on the posterior

No libraries beyond NumPy and Matplotlib. Everything is explicit.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams['figure.figsize'] = (12, 5)
rcParams['font.size'] = 12
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False

np.random.seed(42)

## 1. The RBF Kernel

The squared exponential (RBF) kernel:

$$k(x, x') = \sigma_f^2 \exp\left(-\frac{|x - x'|^2}{2\ell^2}\right)$$

Two hyperparameters:
- **Length-scale** $\ell$: how far apart inputs must be before function values become uncorrelated
- **Signal variance** $\sigma_f^2$: the prior variance of function values

In [ ]:
def rbf_kernel(X1, X2, length_scale=1.0, signal_var=1.0):
    """Compute the RBF (squared exponential) kernel matrix."""
    sqdist = np.sum(X1**2, axis=1).reshape(-1, 1) + \
             np.sum(X2**2, axis=1).reshape(1, -1) - \
             2 * X1 @ X2.T
    return signal_var * np.exp(-0.5 * sqdist / length_scale**2)

## 2. Sampling from the GP Prior

A GP prior with mean $m(x) = 0$ and kernel $k$ says: for any finite set of test points $X_*$, the function values are jointly Gaussian:

$$\mathbf{f}_* \sim \mathcal{N}(\mathbf{0}, K(X_*, X_*))$$

Each sample is one possible function that the GP considers plausible *before seeing any data*.

In [ ]:
X_test = np.linspace(-5, 5, 200).reshape(-1, 1)

K_prior = rbf_kernel(X_test, X_test, length_scale=1.0, signal_var=1.0)
K_prior += 1e-8 * np.eye(len(X_test))  # numerical stability

L_prior = np.linalg.cholesky(K_prior)
prior_samples = L_prior @ np.random.randn(len(X_test), 5)

fig, ax = plt.subplots()
for i in range(5):
    ax.plot(X_test, prior_samples[:, i], alpha=0.7, linewidth=1.5)

ax.fill_between(X_test.ravel(), -2, 2, alpha=0.1, color='steelblue', label='±2σ prior')
ax.axhline(0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_title('GP Prior — 5 sample functions (RBF kernel, ℓ=1.0, σ²_f=1.0)')
ax.legend()
plt.tight_layout()
plt.show()

## 3. GP Posterior — Conditioning on Observations

Given $n$ observations $D_n = \{(x_i, y_i)\}$ with $y_i = f(x_i) + \varepsilon_i$, the posterior at test point $x_*$ is:

$$\mu_n(x_*) = k(x_*, X) [K(X,X) + \sigma_n^2 I]^{-1} \mathbf{y}$$

$$\sigma_n^2(x_*) = k(x_*, x_*) - k(x_*, X) [K(X,X) + \sigma_n^2 I]^{-1} k(X, x_*)$$

In [ ]:
def gp_posterior(X_train, y_train, X_test, length_scale=1.0, signal_var=1.0, noise_var=1e-6):
    """Compute GP posterior mean and variance at test points."""
    K = rbf_kernel(X_train, X_train, length_scale, signal_var) + noise_var * np.eye(len(X_train))
    K_s = rbf_kernel(X_train, X_test, length_scale, signal_var)
    K_ss = rbf_kernel(X_test, X_test, length_scale, signal_var)

    L = np.linalg.cholesky(K)
    alpha = np.linalg.solve(L.T, np.linalg.solve(L, y_train))
    mu = K_s.T @ alpha

    v = np.linalg.solve(L, K_s)
    var = np.diag(K_ss) - np.sum(v**2, axis=0)
    var = np.maximum(var, 1e-10)

    return mu.ravel(), var

## 4. Prior → Posterior: Watching Uncertainty Collapse

We'll observe the same function at 0, 3, and 7 points — exactly matching the three-panel progression from the article.

In [ ]:
true_fn = lambda x: np.sin(x) + 0.5 * np.cos(2.5 * x)

X_obs_all = np.array([-3.5, -2.0, -0.5, 0.8, 2.0, 3.2, 4.0]).reshape(-1, 1)
y_obs_all = true_fn(X_obs_all.ravel())

observation_counts = [0, 3, 7]
titles = ['Prior (n=0)', 'Posterior (n=3)', 'Posterior (n=7, converging)']

fig, axes = plt.subplots(1, 3, figsize=(16, 5), sharey=True)

for idx, (n_obs, title) in enumerate(zip(observation_counts, titles)):
    ax = axes[idx]

    if n_obs == 0:
        mu = np.zeros(len(X_test))
        var = np.ones(len(X_test))
    else:
        X_train = X_obs_all[:n_obs]
        y_train = y_obs_all[:n_obs]
        mu, var = gp_posterior(X_train, y_train, X_test, length_scale=1.0, signal_var=1.0, noise_var=1e-4)

    std = np.sqrt(var)
    ax.fill_between(X_test.ravel(), mu - 2*std, mu + 2*std, alpha=0.2, color='steelblue', label='±2σ')
    ax.plot(X_test, mu, color='steelblue', linewidth=2, label='posterior mean μₙ(x)')
    ax.plot(X_test, true_fn(X_test.ravel()), 'k--', alpha=0.3, linewidth=1, label='true f(x)')

    if n_obs > 0:
        ax.scatter(X_train, y_train, c='black', s=50, zorder=5, label=f'{n_obs} observations')

    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_xlabel('x')
    if idx == 0:
        ax.set_ylabel('f(x)')
    ax.legend(fontsize=8, loc='upper right')

fig.suptitle('GP Prior → Posterior: How Observations Collapse Uncertainty', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

## 5. Effect of Observation Noise

When $\sigma_n^2 > 0$, the posterior doesn't interpolate the data exactly — it smooths through noisy observations.

In [ ]:
noise_levels = [1e-4, 0.1, 0.5]
noise_labels = ['Near-zero noise (σ²_n=0.0001)', 'Moderate noise (σ²_n=0.1)', 'High noise (σ²_n=0.5)']

X_train = X_obs_all
y_noisy = true_fn(X_train.ravel()) + 0.3 * np.random.randn(len(X_train))

fig, axes = plt.subplots(1, 3, figsize=(16, 5), sharey=True)

for ax, noise_var, label in zip(axes, noise_levels, noise_labels):
    mu, var = gp_posterior(X_train, y_noisy, X_test, length_scale=1.0, signal_var=1.0, noise_var=noise_var)
    std = np.sqrt(var)

    ax.fill_between(X_test.ravel(), mu - 2*std, mu + 2*std, alpha=0.2, color='steelblue')
    ax.plot(X_test, mu, color='steelblue', linewidth=2)
    ax.scatter(X_train, y_noisy, c='black', s=50, zorder=5)
    ax.plot(X_test, true_fn(X_test.ravel()), 'k--', alpha=0.3, linewidth=1)
    ax.set_title(label, fontsize=11)
    ax.set_xlabel('x')

axes[0].set_ylabel('f(x)')
fig.suptitle('Effect of Noise Variance on GP Posterior', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

## 6. Computational Cost: O(n³) Matrix Inversion

The bottleneck is the Cholesky decomposition of the $n \times n$ kernel matrix. Let's time it.

In [ ]:
import time

sizes = [50, 100, 200, 500, 1000, 2000, 5000]
times = []

for n in sizes:
    X = np.random.randn(n, 1)
    K = rbf_kernel(X, X) + 1e-6 * np.eye(n)
    start = time.time()
    np.linalg.cholesky(K)
    times.append(time.time() - start)

fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(sizes, times, 'o-', color='steelblue', linewidth=2, markersize=8)
ax.loglog(sizes, [times[0] * (n/sizes[0])**3 for n in sizes], '--', color='gray', alpha=0.5, label='O(n³) reference')
ax.set_xlabel('Number of observations n')
ax.set_ylabel('Time (seconds)')
ax.set_title('Cholesky Decomposition Time — The O(n³) Bottleneck')
ax.legend()
plt.tight_layout()
plt.show()

for n, t in zip(sizes, times):
    print(f'  n={n:>5d}  →  {t:.4f}s')

---

**Next:** [Notebook 2 — Kernel Design & Function Priors](./02_kernels.ipynb) · [Notebook 3 — Acquisition Functions & Full BO Loop](./03_acquisition_functions.ipynb)

**Back to article:** [Bayesian Optimisation — Mathematical Deep Dive](https://omkarray.com/bayesian-optimization.html)